In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from keras.applications import MobileNet
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalMaxPool2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [2]:
import numpy as np

In [3]:
img_rows, img_cols = 224,224
CustomMobileNet = MobileNet(weights="imagenet", include_top=False, input_shape=(img_rows, img_cols, 3))
for layer in CustomMobileNet.layers:
    layer.trainable = False
for (i, layer) in enumerate(CustomMobileNet.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [11]:
def add_top_layer(bottom_layer,num_classes):
    top_layer=bottom_layer.output
    top_layer=GlobalMaxPool2D()(top_layer)
    top_layer=Dense(1024,activation="relu")(top_layer)
    top_layer=Dense(1024,activation="relu")(top_layer)
    top_layer=Dense(512,activation="relu")(top_layer)
    top_layer = Dense(num_classes, activation="softmax")(top_layer)
    return top_layer

In [12]:
train_datagen=ImageDataGenerator(rescale=1/255,
                                shear_range=0.2, 
                                 zoom_range=0.2,
                                 horizontal_flip=True)
train_dataset=train_datagen.flow_from_directory('train',
                                                target_size=(img_rows,img_cols),
                                                batch_size=32,
                                                class_mode='categorical')
       

Found 1098 images belonging to 10 classes.


In [13]:
test_datagen=ImageDataGenerator(rescale=1/255)
test_dataset=test_datagen.flow_from_directory('validation',
                                                target_size=(img_rows,img_cols),
                                                batch_size=32,
                                                class_mode='categorical')

Found 272 images belonging to 10 classes.


In [14]:
num_classes=10
head_model=add_top_layer(CustomMobileNet,num_classes)
model=Model(inputs=CustomMobileNet.input,outputs=head_model)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [15]:
checkpoint=ModelCheckpoint("monkey_breed.h5",
                          save_weights_only=True,
                           monitor='val_loss',
                           mode='min',
                           save_best_only=True)

In [16]:
earlystop = EarlyStopping(monitor="val_loss",
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

callbacks = [earlystop, checkpoint]

In [17]:
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics='accuracy')

In [19]:
model.fit(train_dataset,validation_data=test_dataset,callbacks=callbacks,epochs=10)

Epoch 1/10
35/35 [==============================] - 66s 2s/step - loss: 0.2011 - accuracy: 0.9381 - val_loss: 0.2465 - val_accuracy: 0.9154
Epoch 2/10
35/35 [==============================] - 69s 2s/step - loss: 0.2169 - accuracy: 0.9235 - val_loss: 0.2446 - val_accuracy: 0.9375
Epoch 3/10
35/35 [==============================] - 67s 2s/step - loss: 0.1659 - accuracy: 0.9390 - val_loss: 0.5680 - val_accuracy: 0.8199
Epoch 4/10
35/35 [==============================] - 67s 2s/step - loss: 0.1469 - accuracy: 0.9481 - val_loss: 0.2010 - val_accuracy: 0.9191
Epoch 5/10
35/35 [==============================] - 78s 2s/step - loss: 0.1864 - accuracy: 0.9435 - val_loss: 0.2610 - val_accuracy: 0.9081
Epoch 6/10
35/35 [==============================] - 72s 2s/step - loss: 0.0914 - accuracy: 0.9672 - val_loss: 0.2908 - val_accuracy: 0.9118
Epoch 7/10
35/35 [==============================] - 67s 2s/step - loss: 0.0871 - accuracy: 0.9709 - val_loss: 0.3148 - val_accuracy: 0.8934
Epoch 00007: early s